# Setup

In [10]:
import contextily as cx
import figure_utilities
import statsmodels.api as sm
import constants
from stats_utilities import produce_summary_statistics, select_controls, test_balance
import geopandas as gpd
import matplotlib.pyplot as plt
from panel_utilities import get_value_variable_names, prepare_df_for_DiD
import numpy as np
plt.rcParams['savefig.dpi'] = 300
import statsmodels.api as sm
import os
import pandas as pd

In [11]:
# Store paths.
INPUT_DATA_PANEL = "../data/03_cleaned/crime_analysis_monthly.csv"
INPUT_DATA_TRACTS = "../data/02_intermediate/tracts.csv"
INPUT_DATA_NEIGHBORHOOD_CRIME_COUNTS = "../data/03_cleaned/neighborhood_crime_counts.csv"
INPUT_DATA_NEIGHBORHOODS = "../data/01_raw/2015-2019_neighborhood_tables_2021.12.21.xlsm"
INPUT_DATA_BOSTON_TRACTS_SHAPEFILE = "../data/01_raw/Census_2010_Tracts"
OUTPUT_TABLES = "../output/final_paper/tables"
OUTPUT_FIGURES = "../output/final_paper/figures"

# Summary Statistics

## Map of Evictions, Colored by Poverty Rate in Census Tract

In [12]:
# Read unrestricted dataset into memory.
df = pd.read_csv(INPUT_DATA_PANEL)
df = df.reset_index() # So we can use 'case_number' like a column

# Create spatial data 
unrestricted_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']))
unrestricted_gdf = unrestricted_gdf.set_crs("EPSG:4326")
unrestricted_gdf = unrestricted_gdf.to_crs("EPSG:3857")

# Plot
fig, ax = plt.subplots(figsize=(8, 10))
ax.set_yticklabels([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_xticks([])
unrestricted_gdf.plot(ax=ax,
                      color='black',
                      markersize=0.05)
cx.add_basemap(ax=ax, crs="EPSG:3857", source=cx.providers.CartoDB.Positron)

# Color census tracts by poverty rate.
boston_tracts_gdf = gpd.read_file(INPUT_DATA_BOSTON_TRACTS_SHAPEFILE)[['GEOID10', 'geometry']].set_index('GEOID10')
boston_tracts_gdf.index = boston_tracts_gdf.index.astype(int)
tract_poverty_rates_df = pd.read_csv(INPUT_DATA_TRACTS, usecols=['tract_geoid', 'poor_share2010'],
                                     index_col='tract_geoid')
boston_tracts_gdf = pd.concat([boston_tracts_gdf, tract_poverty_rates_df], axis=1).dropna(
    subset=['geometry', 'poor_share2010']).drop(index=25025990101)
boston_tracts_gdf.plot(ax=ax, column=boston_tracts_gdf['poor_share2010'], cmap='OrRd', alpha=0.4, legend=True,
                       legend_kwds={'label': "Poverty Rate of Census Tract",
                                    'orientation': "horizontal",
                                    'shrink': 0.25})

figure_utilities.save_figure_and_close(fig, os.path.join(OUTPUT_FIGURES, "evictions_map.png"))

## Eviction Filings Over Time

In [13]:
# Plot the number of eviction filings over time.
df.loc[:, 'last_day_of_file_month'] = (pd.to_datetime(df['file_date']) +
                                       pd.tseries.offsets.MonthEnd(0))
filings_per_month = df.groupby('last_day_of_file_month')['case_number'].count()

# Plot eviction filing counts.
fig, ax = plt.subplots()
filings_per_month.plot(ax=ax, kind='line', color='black',
                       zorder=100)
ax.set_ylabel("Number of Evictions")
ax.set_xlabel("Month")
ax.grid(True)
figure_utilities.save_figure_and_close(fig, os.path.join(OUTPUT_FIGURES, "filings_over_time.png"))


In [14]:
# Read unrestricted dataset into memory.
df = pd.read_csv(INPUT_DATA_PANEL)
df = df.reset_index() # So we can use 'case_number' like a column

# Produce summary statistics table.
treatment_date_variable = 'latest_docket_date'
outcomes_of_interest = [f'group_0_crimes_{constants.Analysis.MAIN_RESULTS_RADIUS}m']
summary_statistics_unrestricted, variable_display_names_dict = produce_summary_statistics(df)

# Rename columns.
summary_statistics_unrestricted.index = summary_statistics_unrestricted.index.set_names(["Panel", "Variable"])
column_display_names_dict = {'mean': "Mean", 'std': "S.D.", 'count': "N", '50%': 'Median'}

summary_statistics_unrestricted = summary_statistics_unrestricted.sort_values(['Panel', 'Variable'])
# Keep only outcomes of interest
outcomes = constants.Variables.outcomes.copy()
for outcome in outcomes:
    if outcome not in outcomes_of_interest:
        if f"relative_pre_treatment_change_in_{outcome}" in summary_statistics_unrestricted.index.get_level_values(1):
            summary_statistics_unrestricted = summary_statistics_unrestricted.drop(f"relative_pre_treatment_change_in_{outcome}",
                                                                                   level=1, axis=0)
        if f"pre_treatment_change_in_{outcome}" in summary_statistics_unrestricted.index.get_level_values(1):
            summary_statistics_unrestricted = summary_statistics_unrestricted.drop(f"pre_treatment_change_in_{outcome}",
                                                                                   level=1, axis=0)
        if f"total_twenty_seventeen_{outcome}" in summary_statistics_unrestricted.index.get_level_values(1):
            summary_statistics_unrestricted = summary_statistics_unrestricted.drop(f"total_twenty_seventeen_{outcome}",
                                                                                   level=1, axis=0)

# Drop median column.
summary_statistics_unrestricted = summary_statistics_unrestricted.drop(columns='50%')

# Export to LaTeX.
filename = os.path.join(OUTPUT_TABLES, "summary_statistics.tex")
latex = (summary_statistics_unrestricted
         .rename(index=variable_display_names_dict)
         .rename(columns=column_display_names_dict)
         .style
         .format(formatter={
    'Mean': "{:,.2f}",
    'Median': "{:,.2f}",
    'S.D.': "{:,.2f}",
    'N': "{:,.0f}"})
         .format_index("\\textit{{{}}}", escape="latex", axis=0, level=0)
         .to_latex(None,
                   column_format="llcccc",
                   hrules=True,
                   clines="skip-last;data")).replace("{*}", "{3cm}")
with open(filename, 'w') as file:
    file.write(latex)
summary_statistics_unrestricted

mean  \
Panel                                 Variable                                                           
Panel A: Pre-treatment Outcomes       pre_treatment_change_in_group_0_crimes_250m           -43.568679   
                                      relative_pre_treatment_change_in_group_0_crimes...     -1.356337   
                                      total_twenty_nineteen_group_0_crimes_250_to_300m      110.057731   
                                      total_twenty_nineteen_group_0_crimes_250_to_350m      242.534174   
                                      total_twenty_nineteen_group_0_crimes_250_to_400m      381.027206   
                                      total_twenty_nineteen_group_0_crimes_250m             305.926344   
                                      total_twenty_nineteen_group_0_crimes_300m             415.984074   
                                      total_twenty_nineteen_group_0_crimes_350m             548.465163   
                                      total_twenty_nineteen_group_1_crimes_250_to_300m        8.374917   
                                      total_twenty_nineteen_group_1_crimes_250_to_350m       17.408759   
                                      total_twenty_nineteen_group_1_crimes_250_to_400m       26.331785   
                                      total_twenty_nineteen_group_1_crimes_250m              20.699403   
                                      total_twenty_nineteen_group_1_crimes_300m              29.074320   
                                      total_twenty_nineteen_group_1_crimes_350m              38.108162   
                                      total_twenty_nineteen_group_2_crimes_250_to_300m       19.049104   
                                      total_twenty_nineteen_group_2_crimes_250_to_350m       40.511612   
                                      total_twenty_nineteen_group_2_crimes_250_to_400m       63.805574   
                                      total_twenty_nineteen_group_2_crimes_250m              52.836098   
                                      total_twenty_nineteen_group_2_crimes_300m              71.896483   
                                      total_twenty_nineteen_group_2_crimes_350m              93.370936   
Panel B: Census Tract Characteristics frac_coll_plus2010                                      0.322629   
                                      job_density_2013                                    18066.397982   
                                      med_hhinc2016                                       46685.742535   
                                      poor_share2010                                          0.286367   
                                      popdensity2010                                      23259.527496   
                                      share_white2010                                         0.314273   
Panel C: Case Initiation              for_cause                                               0.140677   
                                      hasAttyD                                                0.028534   
                                      hasAttyP                                                0.897810   
                                      no_cause                                                0.047777   
                                      non_payment                                             0.811546   
Panel D: Case Resolution              case_duration                                          18.398806   
                                      defaulted                                               0.500995   
                                      dismissed                                               0.473789   
                                      heard                                                   0.025216   
                                      judgment                                             1579.941175   

                                                                     

In [15]:
# Share of non entity plaintiffs who are represented by an attorney
df.loc[df['isEntityP'] == 0, 'hasAttyP'].mean()

0.42735042735042733

In [16]:
# Produce treatment timings table.
treatment_timings = (df
                     .groupby(['latest_docket_month', 'judgment_for_plaintiff'])['case_number']
                     .count()
                     .reset_index()
                     .fillna(0))
treatment_timings = treatment_timings.pivot(index='latest_docket_month', columns='judgment_for_plaintiff').fillna(0)
treatment_timings.columns = ["Cases Won By Defendant", "Cases Won By Plaintiff"]
portion_of_all_cases = (treatment_timings['Cases Won By Plaintiff'] + treatment_timings[
    'Cases Won By Defendant']) / len(df)
treatment_timings = pd.concat([treatment_timings, portion_of_all_cases.rename('Portion of All Cases')], axis=1)
sum_across_filing_date = pd.DataFrame(treatment_timings.sum(axis=0)).T
sum_across_filing_date.index = ["All Months"]
treatment_timings = pd.concat([sum_across_filing_date, treatment_timings], axis=0)
treatment_timings.index = treatment_timings.index.rename("Last Docket Date")

# Export to LaTeX.
filename = os.path.join(OUTPUT_TABLES, "treatment_timings.tex")
treatment_timings.style.format(formatter={'Cases Won By Plaintiff': '{:,.0f}',
                                          'Cases Won By Defendant': '{:,.0f}',
                                          'Portion of All Cases': '{:0.2f}'}).to_latex(filename, column_format="lccc",
                                                                                       hrules=True)
treatment_timings

,Cases Won By Defendant,Cases Won By Plaintiff,Portion of All Cases
Last Docket Date,,,
All Months,715.0,792.0,1.000000
2019-06,46.0,13.0,0.039151
2019-07,62.0,42.0,0.069011
2019-08,67.0,87.0,0.102190
2019-09,73.0,92.0,0.109489
2019-10,88.0,83.0,0.113470
2019-11,61.0,63.0,0.082283
2019-12,66.0,65.0,0.086928
2020-01,78.0,123.0,0.133378


In [17]:
# Calculate percent of cases which are in high poverty neighborhoods.
df = df.loc[df['judgment_for_plaintiff'] == 1, :]
original_N = len(df)
cases_in_poor_tracts = len(df.loc[df['poor_share2010'] > 0.20, :])
cases_in_poor_tracts / original_N

0.6439393939393939